In [1]:
from datasets.idda import IDDADataset
from client import Client
from server import Server
from utils.stream_metrics import StreamSegMetrics
from models.deeplabv3 import deeplabv3_mobilenetv2
import datasets.ss_transforms as sstr

In [2]:
class Args:
    def __init__(self,num_rounds,num_epochs,clients_per_round,hnm=False,lr=0.05,bs=4,wd=0,m=0.9):
        #Rounds 
        self.num_rounds=num_rounds
        #Epochs per client for each round
        self.num_epochs=num_epochs
        #Clients selected per round
        self.clients_per_round=clients_per_round
        #Use HardMargin
        self.hnm=hnm
        #Batch size
        self.bs=bs
        #Learning rate
        self.lr=lr
        #Weight decay
        self.wd=wd
        #Momentum
        self.m=m

In [3]:
argsStep1=Args(10,10,10)

metricsStep1 = {
            'eval_train': StreamSegMetrics(16, 'eval_train'),
            'test_same_dom': StreamSegMetrics(16, 'test_same_dom'),
            'test_diff_dom': StreamSegMetrics(16, 'test_diff_dom')
        }
modelStep1 = deeplabv3_mobilenetv2(16)
testTrStep1=sstr.Compose([sstr.RandomResizedCrop((512,928),scale=(0.5,2)),sstr.ToTensor()])
trainTrStep1=sstr.Compose([sstr.RandomResizedCrop((512,928),scale=(0.5,2)),sstr.ToTensor()])

In [4]:
def createCentralizedServer(args,model,metrics,train_transform,test_transform,root='data/idda'):
    iddaTrain=IDDADataset(root,fileName='train.txt',transform=train_transform,client_name='Centralized server')
    iddaTestSame=IDDADataset(root,fileName='test_same_dom.txt',transform=test_transform,client_name='IDDA same domain')
    iddaTestDiff=IDDADataset(root,fileName='test_diff_dom.txt',transform=test_transform,client_name='IDDA different domain')
    train_clients=[Client(args=args,dataset=iddaTrain,model=model)]
    test_clients=[Client(args=args,dataset=iddaTestDiff,model=model,test_client=True),Client(args=argsStep1,dataset=iddaTestSame,model=model,test_client=True)]
    return Server(args=args,train_clients=train_clients,test_clients=test_clients,model=model,metrics=metrics)

In [5]:
server1=createCentralizedServer(args=argsStep1,model=modelStep1,metrics=metricsStep1,train_transform=trainTrStep1,test_transform=testTrStep1)

In [6]:
server1.train()

ROUND 1/10: Training 10 Clients...
CLIENT 1/1: Centralized server
Epoch:0


c:\Users\ollob\anaconda3\envs\mldl23fl_6\lib\site-packages\torchvision\transforms\functional.py:442: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(


Epoch:1


KeyboardInterrupt: 